### Libraries

In [2]:
import pandas as pd
import datetime as dt
from google.colab import files
import io

### Upload the data

In [3]:
#uploading data_raw.csv
uploaded = files.upload()

Saving data_raw.csv to data_raw.csv


# Cleaning

## Primary Cleaning

In [179]:
#reading the file
raw_df = pd.read_csv(io.BytesIO(uploaded['data_raw.csv']), delimiter=';')

#getting a subset with needed columns
raw_df = raw_df[['Date', 'Time UTC', 'Latitude', 'Longitude', 'Depth',
                 'Magnitude Type', 'Magnitude', 'Region Name']]
#convierting all column names to lowercase. Streamlit that we will use creting and app needs 'latitude'/'longitude' to automatically plot the data on the map
raw_df.columns = raw_df.columns.str.lower()

#Replaceing spaces with "_" in column names, it will make accessing the columns easier - by calling df.column1 instead of df['column1']
raw_df.columns = raw_df.columns.str.replace(' ', '_')

#sorting by Date and Time
raw_df = raw_df.sort_values(by=['date','time_utc']).reset_index(drop=True)
raw_df.head()

,date,time_utc,latitude,longitude,depth,magnitude_type,magnitude,region_name
0,2020-01-01,00:01:56,19.22,-67.13,12,Md,2.8,PUERTO RICO REGION
1,2020-01-01,00:02:21,-2.74,127.90,20,M,3.0,"CERAM SEA, INDONESIA"
2,2020-01-01,00:03:38,19.08,-67.09,6,Md,2.5,PUERTO RICO REGION
3,2020-01-01,00:10:43,19.19,-67.84,28,Md,3.1,DOMINICAN REPUBLIC REGION
4,2020-01-01,00:11:23,-25.64,-70.52,53,ML,3.5,"ANTOFAGASTA, CHILE"


## Date and Time_UTC

In [180]:
#creating date_time_utc column that has bot date and time_utc 
raw_df.insert(0,'date_time_utc',pd.to_datetime(raw_df.date.str.cat(raw_df.time_utc,sep=" ")))
#declaring date_time_utc as index so it will be easier to plot timeseries in case we need
#dropping useless columns
raw_df.drop(['date', 'time_utc'], axis=1, inplace=True)

## Magnitude type

In [181]:
#checking unique values 
raw_df.magnitude_type.unique()

array(['Md', ' M', 'ML', 'MD', 'Ml', 'mb', 'Mw', 'ml', 'M ', 'Mc', 'Mh',
       'Mm', 'Mn'], dtype=object)

In [182]:
#Converting all Magnitude types to upper case
raw_df.magnitude_type = [x.upper() for x in raw_df.magnitude_type]
#removing white space in magnitude types in order to be able to group them later on if needed
raw_df.magnitude_type = raw_df.magnitude_type.str.replace(' ', '')
raw_df.magnitude_type.unique()

array(['MD', 'M', 'ML', 'MB', 'MW', 'MC', 'MH', 'MM', 'MN'], dtype=object)

In [183]:
#Saving the Magnitudes descirption for future reference
url_description = "https://www.usgs.gov/natural-hazards/earthquake-hazards/science/magnitude-types?qt-science_center_objects=0#qt-science_center_objects"
description_df_raw = pd.read_html(url_description)
description_df=description_df_raw[0]

In [184]:
description_df.to_csv('magnitudes_description.csv')

## Region name

### Lists for preprocessing

Lists of coutries and US states that will help to map the locations 

In [185]:
Country_list = ['KASHMIR',
  'UNITED STATES',
  'U.S',
 'AFGHANISTAN',
 'ALBANIA',
 'ALGERIA',
 'AMERICAN SAMOA',
 'ANDORRA',
 'ANGOLA',
 'ANGUILLA',
 'ANTARCTICA',
 'ANTIGUA AND BARBUDA',
 'ARGENTINA',
 'ARMENIA',
 'ARUBA',
 'AUSTRALIA',
 'AUSTRIA',
 'AZERBAIJAN',
 'BAHAMAS',
 'BAHRAIN',
 'BANGLADESH',
 'BARBADOS',
 'BELARUS',
 'BELGIUM',
 'BELIZE',
 'BENIN',
 'BERMUDA',
 'BHUTAN',
 'BOLIVIA',
 'BOSNIA AND HERZEGOWINA',
 'BOTSWANA',
 'BOUVET ISLAND',
 'BRAZIL',
 'BRUNEI DARUSSALAM',
 'BULGARIA',
 'BURKINA FASO',
 'BURUNDI',
 'CAMBODIA',
 'CAMEROON',
 'CANADA',
 'CAPE VERDE',
 'CAYMAN ISLANDS',
 'CENTRAL AFRICAN REP',
 'CHAD',
 'CHILE',
 'CHINA',
 'CHRISTMAS ISLAND',
 'COCOS ISLANDS',
 'COLOMBIA',
 'COMOROS',
 'CONGO',
 'COOK ISLANDS',
 'COSTA RICA',
 'COTE D`IVOIRE',
 'CROATIA',
 'CUBA',
 'CYPRUS',
 'CZECH REPUBLIC',
 'DENMARK',
 'DJIBOUTI',
 'DOMINICA',
 'DOMINICAN REPUBLIC',
 'EAST TIMOR',
 'ECUADOR',
 'EGYPT',
 'EL SALVADOR',
 'EQUATORIAL GUINEA',
 'ERITREA',
 'ESTONIA',
 'ETHIOPIA',
 'FALKLAND ISLANDS (MALVINAS)',
 'FAROE ISLANDS',
 'FIJI',
 'FINLAND',
 'FRANCE',
 'FRENCH GUIANA',
 'FRENCH POLYNESIA',
 'FRENCH S. TERRITORIES',
 'GABON',
 'GAMBIA',
 'GEORGIA',
 'GERMANY',
 'GHANA',
 'GIBRALTAR',
 'GREECE',
 'GREENLAND',
 'GRENADA',
 'GUADELOUPE',
 'GUAM',
 'GUATEMALA',
 'GUINEA',
 'GUINEA-BISSAU',
 'GUYANA',
 'HAITI',
 'HONDURAS',
 'HONG KONG',
 'HUNGARY',
 'ICELAND',
 'INDIA',
 'INDONESIA',
 'IRAN',
 'IRAQ',
 'IRELAND',
 'ISRAEL',
 'ITALY',
 'JAMAICA',
 'JAPAN',
 'JORDAN',
 'KAZAKHSTAN',
 'KENYA',
 'KIRIBATI',
 'KOREA (NORTH)',
 'KOREA (SOUTH)',
 'KUWAIT',
 'KYRGYZSTAN',
 'LAOS',
 'LATVIA',
 'LEBANON',
 'LESOTHO',
 'LIBERIA',
 'LIBYA',
 'LIECHTENSTEIN',
 'LITHUANIA',
 'LUXEMBOURG',
 'MACAU',
 'MACEDONIA',
 'MADAGASCAR',
 'MALAWI',
 'MALAYSIA',
 'MALDIVES',
 'MALI',
 'MALTA',
 'MARSHALL ISLANDS',
 'MARTINIQUE',
 'MAURITANIA',
 'MAURITIUS',
 'MAYOTTE',
 'MEXICO',
 'MICRONESIA',
 'MOLDOVA',
 'MONACO',
 'MONGOLIA',
 'MONTSERRAT',
 'MOROCCO',
 'MOZAMBIQUE',
 'MYANMAR',
 'NAMIBIA',
 'NAURU',
 'NEPAL',
 'NETHERLANDS',
 'NETHERLANDS ANTILLES',
 'NEW CALEDONIA',
 'NEW ZEALAND',
 'NICARAGUA',
 'NIGER',
 'NIGERIA',
 'NIUE',
 'NORFOLK ISLAND',
 'NORTHERN MARIANA ISLANDS',
 'NORWAY',
 'OMAN',
 'PAKISTAN',
 'PALAU',
 'PANAMA',
 'PAPUA NEW GUINEA',
 'PARAGUAY',
 'PERU',
 'PHILIPPINES',
 'PITCAIRN',
 'POLAND',
 'PORTUGAL',
 'PUERTO RICO',
 'QATAR',
 'REUNION',
 'ROMANIA',
 'RUSSIAN FEDERATION',
 'RUSSIA',
 'RWANDA',
 'SAINT KITTS AND NEVIS',
 'SAINT LUCIA',
 'ST VINCENT/GRENADINES',
 'SAMOA',
 'SAN MARINO',
 'SAO TOME',
 'SAUDI ARABIA',
 'SENEGAL',
 'SEYCHELLES',
 'SIERRA LEONE',
 'SINGAPORE',
 'SLOVAKIA',
 'SLOVENIA',
 'SOLOMON ISLANDS',
 'SOMALIA',
 'SOUTH AFRICA',
 'SPAIN',
 'SRI LANKA',
 'ST. HELENA',
 'ST.PIERRE',
 'SUDAN',
 'SURINAME',
 'SWAZILAND',
 'SWEDEN',
 'SWITZERLAND',
 'SYRIA',
 'TAIWAN',
 'TAJIKISTAN',
 'TANZANIA',
 'THAILAND',
 'TOGO',
 'TOKELAU',
 'TONGA',
 'TRINIDAD AND TOBAGO',
 'TUNISIA',
 'TURKEY',
 'TURKMENISTAN',
 'TUVALU',
 'UGANDA',
 'UKRAINE',
 'UNITED ARAB EMIRATES',
 'UNITED KINGDOM',
 'U.K.',
 'URUGUAY',
 'UZBEKISTAN',
 'VANUATU',
 'VATICAN CITY STATE',
 'VENEZUELA',
 'VIET NAM',
 'VIRGIN ISLANDS (BRITISH)',
 'VIRGIN ISLANDS (U.S.)',
 'WESTERN SAHARA',
 'YEMEN',
 'YUGOSLAVIA',
 'ZAIRE',
 'ZAMBIA',
 'ZIMBABWE']

In [186]:
US_states_list = ['ALASKA',
  'ISLAND OF HAWAII, HAWAII',
 'ALABAMA',
 'ARKANSAS',
 'AMERICAN SAMOA',
 'ARIZONA',
 'CALIFORNIA',
 'COLORADO',
 'CONNECTICUT',
 'DISTRICT OF COLUMBIA',
 'DELAWARE',
 'DAKOTA',
 'FLORIDA',
 'GUAM',
 'HAWAII',
 'IOWA',
 'IDAHO',
 'ILLINOIS',
 'INDIANA',
 'KANSAS',
 'KENTUCKY',
 'LOUISIANA',
 'MASSACHUSETTS',
 'MARYLAND',
 'MAINE',
 'MICHIGAN',
 'MINNESOTA',
 'MISSOURI',
 'NORTHERN MARIANA ISLANDS',
 'MISSISSIPPI',
 'MONTANA',
 'NATIONAL',
 'NORTH CAROLINA',
 'NORTH DAKOTA',
 'NEBRASKA',
 'NEW HAMPSHIRE',
 'NEW JERSEY',
 'NEW MEXICO',
 'NEVADA',
 'NEW YORK',
 'OHIO',
 'OKLAHOMA',
 'OREGON',
 'PENNSYLVANIA',
 'PUERTO RICO',
 'RHODE ISLAND',
 'SOUTH CAROLINA',
 'SOUTH DAKOTA',
 'TENNESSEE',
 'TEXAS',
 'UTAH',
 'VIRGINIA',
 'VIRGIN ISLANDS',
 'VERMONT',
 'WASHINGTON',
 'WISCONSIN',
 'WEST VIRGINIA',
 'WYOMING']

### Wrangling

In [187]:
#raw_df.region_name.sort_values().unique() #check for the unique values
len(raw_df.region_name.unique())

867

In [188]:
#replacing region_names that contains certain str(e.g 'CALIF ') in to 'CALIFORNIA' 
#we doing it now, since these two regions are U.S. states and not a country
raw_df.region_name = raw_df.region_name.str.replace(r'(^.*CALIF.*$)', 'CALIFORNIA')
raw_df.region_name = raw_df.region_name.str.replace(r'(^.*TENNESSEE.*$)', 'TENNESSEE')

#Creating a list that will consists of all reagions except SEAs and BORDERs, since they can belong to more than 1 country
region_final_Countries_replaced = []
for i in raw_df.region_name:
  if "BORDER" or 'SEA' not in i:
    for j in Country_list:
      if j in i:
        i=j
    region_final_Countries_replaced.append(i)
  else:
    region_final_Countries_replaced.append(i)

#Rename State regions to the US
for i in US_states_list:
  for j, item in enumerate(region_final_Countries_replaced):
    if i in item:
      region_final_Countries_replaced[j] = 'U.S.'

raw_df['Countries'] = region_final_Countries_replaced

#Here we manually handle exeptions and typos
raw_df.Countries = raw_df.Countries.str.replace(r'(^.*KAMCHATKA.*$)', 'RUSSIA')
raw_df.Countries = raw_df.Countries.str.replace(r'(^.*FRANZ JOSEF LAND.*$)', 'RUSSIA')
raw_df.Countries = raw_df.Countries.str.replace(r'(^.*SEVERNAYA ZEMLYA.*$)', 'RUSSIA')
raw_df.Countries = raw_df.Countries.str.replace(r'(^.*KURIL.*$)', 'RUSSIA')
raw_df.Countries = raw_df.Countries.str.replace(r'(^.*P.N.G..*$)', 'PAPUA NEW GUINIA')
raw_df.Countries = raw_df.Countries.str.replace(r'(^.*MARIANA.*$)', 'MARIANA ISLANDS')
raw_df.Countries = raw_df.Countries.str.replace(r'(^.*ALEUTIAN.*$)', 'ALEUTIAN ISLANDS')
raw_df.Countries = raw_df.Countries.str.replace(r'(^.*N.Z..*$)', 'NEW ZEALAND')
raw_df.Countries = raw_df.Countries.str.replace(r'(^.*KERMADEC.*$)', 'NEW ZEALAND')
raw_df.Countries = raw_df.Countries.str.replace(r'(^.*CAN..*$)', 'CANADA')
raw_df.Countries = raw_df.Countries.str.replace(r'(^.*QUEEN CHARLOTTE.*$)', 'CANADA')
raw_df.Countries = raw_df.Countries.str.replace(r'(^.*MEX..*$)', 'MEXICO')
raw_df.Countries = raw_df.Countries.str.replace(r'(^.*REVILLA GIGEDO.*$)', 'MEXICO')
raw_df.Countries = raw_df.Countries.str.replace(r'(^.*GALAPAGOS.*$)', 'GALAPAGOS ISLANDS')
raw_df.Countries = raw_df.Countries.str.replace(r'(^.*ISLAND REGION.*$)', 'ISLAND')
raw_df.Countries = raw_df.Countries.str.replace(r'(^.*TANGANYIKA.*$)', 'CONGO')
raw_df.Countries = raw_df.Countries.str.replace(r'(^.*MALDIVE.*$)', 'MALDIVE ISLANDS')
raw_df.Countries = raw_df.Countries.str.replace(r'(^.*BARBUDA.*$)', 'ANTIGUA AND BARBUDA')
raw_df.Countries = raw_df.Countries.str.replace(r'(^.*ASCENSION.*$)', 'ASCENSION ISLAND')
raw_df.Countries = raw_df.Countries.str.replace(r'(^.*HONSHU.*$)', 'JAPAN')
raw_df.Countries = raw_df.Countries.str.replace(r'(^.*MACQUARIE.*$)', 'AUSTRALIA')
raw_df.Countries = raw_df.Countries.str.replace(r'(^.*SVALBARD.*$)', 'NORWAY')
raw_df.Countries = raw_df.Countries.str.replace(r'(^.*SUMATRA.*$)', 'INDONASIA')
raw_df.Countries = raw_df.Countries.str.replace(r'(^.*PHILIPPINE.*$)', 'PHILIPPINES')
raw_df.Countries = raw_df.Countries.str.replace(r'(^.*PRINCE EDWARD ISLANDS.*$)', 'SOUTH AFRICA')
raw_df.Countries = raw_df.Countries.str.replace(r'(^.*TIMOR.*$)', 'TIMOR')
raw_df.Countries = raw_df.Countries.str.replace(r'(^.*TRINIDAD-TOBAGO.*$)', 'TRINIDAD-TOBAGO')
raw_df.Countries = raw_df.Countries.str.replace(r'(^.*UNITED KINGDOM.*$)', 'U.K.')
raw_df.loc[raw_df.region_name == 'SOUTH GEORGIA ISLAND REGION', 'Countries'] = 'U.K.'
raw_df.loc[raw_df.region_name == 'GEORGIA, USA', 'Countries'] = 'U.S.'

In [189]:
len(raw_df.Countries.sort_values().unique())

233

## Saving clean data

In [190]:
raw_df.to_csv('data_clean.csv')